# Utils

Cells in this notebook can be used to perform scriptable tasks like generating pages based on data structures. Normally these would be implemented as Jekyll plugins, but GitHub Pages does not allow custom plugins. There's probably a more clever way to run these tasks in GitHub workflows or something, but this works fine.

## Generate Tag Pages

The following cell reads `_config.yml` and all collection files, extracts tags, and writes a `.md` file for each tag under `tags/` as `tags/<tag>.md`.

Run this cell whenever new tags are added.


In [5]:
import os
import re
import glob
import yaml

# Load Jekyll configuration
with open('_config.yml', 'r') as f:
    cfg = yaml.safe_load(f)

# Determine collections (including posts)
collections = set(cfg.get('collections', {}).keys())
collections.add('posts')

# Gather tags from all collection files
tags = set()
for coll in collections:
    dir_name = '_posts' if coll == 'posts' else f"_{coll}"
    if not os.path.isdir(dir_name):
        continue
    for path in glob.glob(os.path.join(dir_name, '*.md')):
        with open(path, 'r') as f:
            content = f.read()
        fm = re.match(r'^---\s*(.*?)\s*---', content, re.DOTALL)
        if not fm:
            continue
        data = yaml.safe_load(fm.group(1))
        for t in data.get('tags', []) or []:
            if isinstance(t, str):
                t = t.strip()
                tags.add(t)

# Create tags directory and write tag pages
os.makedirs('_tags', exist_ok=True)
for tag in tags:
    slug = re.sub(r'[^\w-]', '', tag.lower().replace(' ', '-'))
    header = f"""---
layout: tag
tag: {tag}
title: {tag}
---"""
    with open(os.path.join('_tags', f'{slug}.md'), 'w') as f:
        f.write(header + '\n')
print(f"Generated {len(tags)} tag pages in '_tags/' directory.")

Generated 93 tag pages in '_tags/' directory.


In [ ]:
#%pip install pyyaml


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Generate Markdown File for Each Image File

This cell generates a corresponding markdown file for each image in a directory (drawings or stereoscopic-images etc.) and puts it in the corresponding Jekyll collection directory with the same name, just with the underscore prefix. If the markdown file already exists, it will _not_ be overwritten.

In [44]:
import os
import re
import sys
from pathlib import Path

def generate_md_for_images(directory, dest_dir, layout):
    img_pattern = re.compile(r'^(?P<date>\d{4}-\d{2}-\d{2})_(?P<name>.+)\.(?:jpg|png|gif)$', re.IGNORECASE)
    directory = Path(directory)
    # if the source directory doesn't exist, exit
    if not directory.exists():
        print(f"Source directory {directory} does not exist.")
        return
    dest_dir = Path(dest_dir)
    # if the destination directory doesn't exist, create it
    if not dest_dir.exists():
        dest_dir.mkdir(parents=True)
    for img_path in directory.iterdir():
        if not img_path.is_file():
            continue
        m = img_pattern.match(img_path.name)
        if not m:
            continue
        date = m.group('date')
        name = m.group('name')
        filename = img_path.name
        pagename = f"{date}_{name}"
        md_path = dest_dir / f"{pagename}.md"
        if md_path.exists():
            continue
        front_matter = f"""---
layout: {layout}
filename: {filename}
pagename: {pagename}
date: {date}
tags:
---\n"""
        md_path.write_text(front_matter)
        print(f"Created {md_path}")

directory = 'drawings/'
dest_dir = '_drawings/'
layout = 'drawing'
generate_md_for_images(directory, dest_dir, layout)

directory = 'stereoscopic-images/'
dest_dir = '_stereoscopic_images/'
layout = 'stereoscopic_image'
generate_md_for_images(directory, dest_dir, layout)


Created _drawings/2025-11-24_wave-threshold.md


In [30]:
# find images with the name pattern YYYY-MM-DD.jpg and rename them to YYYY-MM-DD_1.jpg, YYYY-MM-DD_2.jpg, etc.
def rename_images(directory):
    # pattern is YYYY-MM-DD.jpg
    img_pattern = re.compile(r'^(?P<date>\d{4}-\d{2}-\d{2})\.jpg$', re.IGNORECASE)
    directory = Path(directory)
    for img_path in directory.iterdir():
        if not img_path.is_file():
            continue
        m = img_pattern.match(img_path.name)
        if not m:
            continue
        date = m.group('date')
        name = '1'
        new_name = f"{date}_{name}.jpg"
        new_path = directory / new_name
        if new_path.exists():
            continue
        img_path.rename(new_path)
        print(f"Renamed {img_path} to {new_path}")

rename_images('stereoscopic-images/')
rename_images('drawings/')

# Convert JPGs to SVGs

In [45]:
# Batch JPG/PNG/TIF → SVG vectorization for black/white line art
# Tools required: ImageMagick and Potrace
#   macOS:    brew install imagemagick potrace
#   Ubuntu:   sudo apt-get update && sudo apt-get install -y imagemagick potrace
#   Windows:  winget install ImageMagick.ImageMagick ; winget install potrace.potrace

from pathlib import Path
import subprocess, os, tempfile, concurrent.futures
from typing import List
from dataclasses import dataclass
from tqdm import tqdm  # pip install tqdm

@dataclass
class Config:
    input_dir: Path
    output_dir: Path
    extensions: List[str] = None     # ["jpg","jpeg","png","tif","tiff","bmp"]
    threshold: int = 88              # 0–100; raise to clean paper, lower if lines break
    despck: int = 0                  # pre-despeckle min area (pixels) via ImageMagick; 0 disables
    turdsize: int = 2                # Potrace speckle filter (area in px)
    alphamax: float = 1.0            # curve smoothness (lower=sharper)
    opttol: float = 0.2              # curve optimization tolerance
    turnpolicy: str = "minority"     # black|white|left|right|minority|majority
    overwrite: bool = False
    jobs: int = max(os.cpu_count() or 4, 4)

    def __post_init__(self):
        if self.extensions is None:
            self.extensions = ["jpg","jpeg","png","tif","tiff","bmp"]
        self.input_dir = Path(self.input_dir)
        self.output_dir = Path(self.output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)


In [46]:
def _run(cmd: list):
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}\n{p.stderr}")
    return p

def _find_magick() -> str:
    # Prefer "magick" to avoid conflicts with other "convert" binaries
    for candidate in ["magick", "convert"]:
        try:
            subprocess.run([candidate, "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            return candidate
        except Exception:
            continue
    raise RuntimeError("ImageMagick not found. Install it and ensure 'magick' or 'convert' is on PATH.")

def _ensure_tools():
    _ = _find_magick()
    try:
        subprocess.run(["potrace", "--version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    except Exception:
        raise RuntimeError("Potrace not found. Install it and ensure 'potrace' is on PATH.")

def _to_pbm(src: Path, pbm_path: Path, cfg: Config):
    magick = _find_magick()
    cmd = [
        magick, str(src),
        "-colorspace", "Gray",
        "-filter", "point",      # no antialias
        "-resize", "100%",
        "-threshold", f"{cfg.threshold}%",
    ]
    if cfg.despck > 0:
        cmd += [
            "-define", f"connected-components:area-threshold={cfg.despck}",
            "-define", "connected-components:mean-color=true",
            "-connected-components", "4",
            "-auto-level"
        ]
    cmd += ["-type", "bilevel", "-compress", "none", str(pbm_path)]
    _run(cmd)

def _potrace_to_svg(pbm_path: Path, svg_path: Path, cfg: Config):
    cmd = [
        "potrace", str(pbm_path),
        "--svg",
        "--output", str(svg_path),
        "--turdsize", str(cfg.turdsize),
        "--alphamax", str(cfg.alphamax),
        "--opttolerance", str(cfg.opttol),
        "--flat",
        "--longcurve",
        "--group",
        # "--tight", # This option crops the image to smallest size, which messes up the image dimensions
        "--turnpolicy", cfg.turnpolicy,
    ]
    _run(cmd)

def vectorize_one(src: Path, dst_dir: Path, cfg: Config) -> str:
    dst = dst_dir / (src.stem + ".svg")
    if dst.exists() and not cfg.overwrite:
        return f"skip  {src.name}"
    with tempfile.TemporaryDirectory() as td:
        pbm = Path(td) / (src.stem + ".pbm")
        _to_pbm(src, pbm, cfg)
        _potrace_to_svg(pbm, dst, cfg)
    return f"done  {src.name} → {dst.name}"

def discover_inputs(root: Path, extensions: List[str]) -> List[Path]:
    exts = {e.lower().lstrip(".") for e in extensions}
    files = []
    for ext in exts:
        files.extend(root.rglob(f"*.{ext}"))
        files.extend(root.rglob(f"*.{ext.upper()}"))
    # de-dup while preserving order
    seen = set()
    out = []
    for f in files:
        if f.resolve() not in seen:
            seen.add(f.resolve())
            out.append(f)
    return out

def batch_vectorize(cfg: Config):
    _ensure_tools()
    files = discover_inputs(cfg.input_dir, cfg.extensions)
    if not files:
        print("No input files found.")
        return
    print(f"Found {len(files)} images. Writing SVGs to: {cfg.output_dir}")
    with concurrent.futures.ThreadPoolExecutor(max_workers=cfg.jobs) as ex:
        futs = {ex.submit(vectorize_one, f, cfg.output_dir, cfg): f for f in files}
        for fut in tqdm(concurrent.futures.as_completed(futs), total=len(futs)):
            try:
                _ = fut.result()
            except Exception as e:
                print(f"FAIL  {futs[fut].name}: {e}")


In [47]:
# Edit paths and knobs, then run this cell.
cfg = Config(
    input_dir="drawings",       # folder containing your JPG/PNG/TIF
    output_dir="svgs",          # output folder for SVGs
    extensions=["jpg","jpeg","png","tif","tiff","bmp"],
    threshold=88,               # raise to clean paper; lower if lines break
    despck=0,                   # try 16 or 32 if dust/specks
    turdsize=2,                 # Potrace speck filter
    alphamax=1.0,               # 0.6–0.8 for crisper corners
    opttol=0.2,
    turnpolicy="minority",
    overwrite=False,
    jobs=max(os.cpu_count() or 4, 4),
)

# # Optional stricter cleanup
# cfg.threshold = 90
# cfg.despck = 16
# cfg.alphamax = 0.8
# cfg.turdsize = 4

# # Optional preserve thin lines
# cfg.threshold = 82
# cfg.despck = 0
# cfg.alphamax = 1.0
# cfg.turdsize = 2

batch_vectorize(cfg)


Found 688 images. Writing SVGs to: svgs


100%|██████████| 688/688 [00:01<00:00, 583.11it/s]


# Generate PNGs from SVGs

In [34]:
from pathlib import Path
import subprocess, os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def svg_to_png_batch(input_dir, output_dir, dpi=600, overwrite=False, jobs=None):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    svgs = list(input_dir.rglob("*.svg"))
    if not svgs:
        print("No SVGs found.")
        return

    magick = "magick" if os.name == "nt" else "convert"
    jobs = jobs or os.cpu_count() or 4

    def convert_one(svg):
        png = output_dir / (svg.stem + ".png")
        if png.exists() and not overwrite:
            return f"skip  {svg.name}"
        cmd = [
            magick, str(svg),
            "-density", str(dpi),
            "-background", "white",
            "-flatten",
            "-units", "PixelsPerInch",
            str(png)
        ]
        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        return f"done  {svg.name} → {png.name}"

    print(f"Converting {len(svgs)} SVGs to PNG at {dpi} DPI using {jobs} workers.")
    with ThreadPoolExecutor(max_workers=jobs) as ex:
        for msg in tqdm(ex.map(convert_one, svgs), total=len(svgs)):
            print(msg)

In [35]:
# Example usage
svg_to_png_batch("svgs", "pngs-for-book", dpi=300)

Converting 677 SVGs to PNG at 300 DPI using 8 workers.


  0%|          | 0/677 [00:00<?, ?it/s]

skip  2024-09-11_tight-rope.svg
skip  2025-07-22_angels.svg
skip  2024-01-18_onion-face.svg
skip  2024-08-16_yin-yang.svg
skip  2024-07-18_owl-ring.svg
skip  2024-06-21_time-perspective.svg
skip  2024-03-23_spider.svg
skip  2025-04-18_star-face.svg
skip  2024-04-08_double-vision.svg
skip  2024-11-20_1.svg
skip  2025-08-20_shape-faces.svg
skip  2025-05-11_arch.svg
skip  2024-10-21_upward-spiral.svg
skip  2025-02-22_septagram-accurate.svg
skip  2023-11-22_triangle-faces.svg
skip  2024-04-12_triangular.svg
skip  2024-11-10_fizz-buzz.svg
skip  2024-06-06_balloon-flower.svg
skip  2025-02-16_paper-airplane-bootstrap.svg
skip  2025-10-24_ribs.svg
skip  2025-02-18_pool-face.svg
skip  2025-02-06_hand-wiggle-thing.svg
skip  2024-04-30_mind-network.svg
skip  2024-09-28_circle-dot-head-loop.svg
skip  2024-09-09_doodles.svg
skip  2023-12-24_recursive-quadrant.svg
skip  2025-11-09_eyes.svg
skip  2024-09-02_time-check.svg
skip  2024-04-09_glasses.svg
skip  2024-02-28_big-face.svg
skip  2025-07-06_sli

100%|██████████| 677/677 [00:00<00:00, 738.62it/s]

done  2025-11-13_sun-and-crescents.svg → 2025-11-13_sun-and-crescents.png
skip  2023-12-05_good-view.svg
skip  2024-02-17_planets.svg
skip  2024-10-13_octo-heads.svg
skip  2025-09-23_middle-quadrant.svg
skip  2025-01-20_two-hills.svg
skip  2024-06-24_orbs.svg
skip  2025-08-14_grid.svg
skip  2025-05-02_box.svg
skip  2025-04-16_rearranging.svg
skip  2025-06-19_face-box-box-face.svg
skip  2024-02-02_textures.svg
skip  2025-07-21_mountains.svg
skip  2025-08-07_rainclouds.svg
skip  2025-03-16_hands-ground-globe-sky.svg
skip  2025-03-21_chairface.svg
skip  2024-02-16_rotating.svg
skip  2024-01-31_bug-box-hug.svg
skip  2025-10-20_double-figure-8.svg
skip  2025-10-11_shaggy-wobble-face.svg
skip  2025-01-22_perspective-cube.svg
skip  2024-01-01_swimming.svg
skip  2024-10-11_coffee-duck.svg
skip  2025-10-10_door-to-door.svg
skip  2025-07-26_doors.svg
skip  2025-04-25_progression.svg
skip  2025-07-03_tubes.svg
skip  2024-07-16_globe-faces.svg
skip  2025-10-31_4d-pizza-house.svg
skip  2025-03-09_c